In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# gensim으로 네이버 기사 토픽 모델링

In [2]:
!pip install gensim

In [3]:
!pip install konlpy

In [4]:
%%shell
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [5]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [6]:
%%shell
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141807 sha256=ed031aa61666a6bc2f86652afa9ff98cf59f073842f8cb16ad12acd5c6546087
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
  

In [7]:
from tqdm.notebook import tqdm
from konlpy.tag import Mecab
import string
from gensim import corpora, models

import numpy as np
import re
import pickle
import matplotlib.pyplot as plt

## 텍스트 전처리 함수

In [8]:
def read_documents(input_file_name):

  corpus = []
  with open(input_file_name, 'rb') as f:
    temp_corpus = pickle.load(f)

  for page in temp_corpus:
    corpus += page

  return corpus


def text_cleaning(docs):

    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):
  SW = set()

  for i in string.punctuation:
    SW.add(i)

  with open(path) as f:
    for word in f:
      SW.add(word)

  return SW

def text_tokenizing(corpus, tokenizer):
  mecab = Mecab()
  token_corpus = []

  if tokenizer == 'noun':
    for n in tqdm(range(len(corpus)), desc="Preprocessing"):
      token_text = mecab.nouns(corpus[n])
      token_text = [word for word in token_text if word not in SW and len(word) > 1]
      token_corpus.append(token_text)

  elif tokenizer == 'morph':
    for n in tqdm(range(len(corpus)), desc='Preprocessing'):
      token_text = mecab.morphs(corpus[n])
      token_text = [word for word in token_text if word not in SW and len(word) > 1]
      token_corpus.append(token_text)

  elif tokenizer == 'word':
    for n in tqdm(range(len(corpus)), desc='Preprocessing'):
      token_text = corpus[n].split()
      token_text = [word for word in token_text if word not in SW and len(word) > 1]
      token_corpus.append(token_text)

  return token_corpus

In [9]:
input_file_name = '/content/drive/MyDrive/DL/AtoZ/crawling/naver_news_content.pk'
documents = read_documents(input_file_name)
SW = define_stopwords('/content/drive/MyDrive/DL/AtoZ/data/stopwords-ko.txt')
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer='noun')

Preprocessing:   0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '데이터', '뉴스', '범죄', '양상', '코로나', '범죄', '데이터', '분석', '사이버', '범죄', '직거래', '피해', '성인', '착취', '연평균', '감소', '사이버', '음란물', '아동', '대상', '남성', '남성', '코로나', '캠핑', '수요', '지난해', '온라인', '거래', '사이트', '고가', '캠핑', '용품', '판매', '구입', '거래', '대금', '인터넷', '사람', '장비', '사진', '도용', '자신', '이름', '날짜', '합성', '수법', '이후', '구입', '택배', '송장', '번호', '택배', '취소', '상자', '방식', '전형', '사이버', '사기', '코로나', '이후', '사이버', '범죄', '코로나', '사회', '거리', '두기', '기간', '동안', '폭력', '절도', '강력범', '사이버', '사기', '사이버', '금융', '범죄', '온라인', '범죄', '증가', '서울', '신문', '경찰청', '공공', '데이터', '포털', '공개', '연도', '사이버', '범죄', '현황', '전국', '경찰서', '강력범', '발생', '현황', '프로그래밍', '언어', '파이썬', '이용', '분석', '결과', '강력범', '살인', '강도', '폭력', '감소', '사이버', '범죄', '이상', '연평균', '포인트', '사이버', '범죄', '증가', '코로나', '발생', '이전', '대비', '포인트', '증가', '강력범', '사이버', '범죄', '합산', '전체', '범죄', '발생', '건수', '차이', '범죄', '발생', '공간', '오프라인', '온라인', '이동', '파악', '보이스', '피싱', '사이버', '금융', '범죄', '검거율', '사이버', '범죄', '발생', '범죄', '사이버', '사기', '범죄', '사이버',

## 토픽 모델링에 사용할 함수 확인

In [11]:
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [12]:
print(dictionary)

Dictionary(986 unique tokens: ['가능', '가지', '감소', '강도', '강력범']...)


In [13]:
corpus[0][:5]

[(0, 1), (1, 1), (2, 8), (3, 1), (4, 5)]

In [14]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.010046627803345138),
 (1, 0.025246588893817586),
 (2, 0.2019727111505407),
 (3, 0.025246588893817586),
 (4, 0.12623294446908792)]

In [15]:
# LDA model 만들기

model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [16]:
# LDA 결과 확인
model.show_topic(0, 10)

[('교육', 0.013311612),
 ('시험', 0.013040447),
 ('사이버', 0.012972998),
 ('활용', 0.0118769705),
 ('범죄', 0.011242759),
 ('플레이어', 0.006750705),
 ('데이터', 0.0066957204),
 ('내용', 0.006665112),
 ('과정', 0.006648307),
 ('음란물', 0.006518966)]

## 토픽 모델링을 추가한 함수

In [17]:
NUM_TOPICS = 3
NUM_TOPIC_WORDS = 30

In [18]:
def build_doc_term_mat(documents):
  print('Building documents-term matrix.')
  dictionary = corpora.Dictionary(documents)
  corpus = [dictionary.doc2bow(document) for document in documents]
  
  return corpus, dictionary

def print_topic_words(model):
  print('\nPrinting topic words.\n')

  for topic_id in range(model.num_topics):
    topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
    print(f'Topic ID: {topic_id}')

    for topic_word, prob in topic_word_probs:
      print(f'\t{topic_word}\t{prob}')

    print('\n')

In [19]:
corpus, dictionary = build_doc_term_mat(tokenized_text)

Building documents-term matrix.


In [20]:
model = models.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha='auto', eta='auto')

In [21]:
print_topic_words(model)


Printing topic words.

Topic ID: 0
	교육	0.021231412887573242
	사이버	0.011844931170344353
	범죄	0.010181126184761524
	과정	0.009821266867220402
	데이터	0.009466580115258694
	분석	0.008849953301250935
	디지털	0.00841989740729332
	활용	0.008173519745469093
	시험	0.006768717430531979
	진행	0.0067635709419846535
	기업	0.006564157549291849
	본문	0.005883006379008293
	버스	0.005669322330504656
	음란물	0.00550803542137146
	발생	0.005505559500306845
	전문가	0.005501959472894669
	플레이어	0.0052671353332698345
	가능	0.005116957239806652
	기술	0.004932123702019453
	증가	0.00484571885317564
	내용	0.004829373210668564
	합병	0.004661059472709894
	청담러닝	0.0045779999345541
	대표	0.004521168302744627
	추가	0.004453002475202084
	코로나	0.004421600140631199
	프로그램	0.004328207112848759
	영어	0.004182425327599049
	온라인	0.004112915135920048
	건수	0.004093823488801718


Topic ID: 1
	시험	0.0186888687312603
	활용	0.013858827762305737
	교육	0.012143670581281185
	데이터	0.009344853460788727
	내용	0.008622284047305584
	자격증	0.007543736603111029
	과정	0.007003456354141235
	기업	0.006300885

## pyLDAvis를 통한 토픽 모델링 결과 시각화

In [22]:
!pip install pyLDAvis

In [25]:
import pyLDAvis
import pyLDAvis.gensim_models

In [26]:
pyLDAvis.enable_notebook()

In [27]:
data = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
data

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.017898  0.012539       1        1  37.207807
1      0.025976  0.003616       2        1  31.702070
2     -0.008078 -0.016155       3        1  31.090122, topic_info=    Term       Freq      Total Category  logprob  loglift
702   시험  30.000000  30.000000  Default  30.0000  30.0000
71   사이버  18.000000  18.000000  Default  29.0000  29.0000
60    범죄  15.000000  15.000000  Default  28.0000  28.0000
821   학교   6.000000   6.000000  Default  27.0000  27.0000
564   활용  28.000000  28.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
32    내용   3.590572  17.553100   Topic3  -5.5867  -0.4186
609   역량   3.036181  10.306973   Topic3  -5.7545  -0.0539
0     가능   2.933485  11.340245   Topic3  -5.7889  -0.1839
65    분석   2.998327  16.793540   Topic3  -5.7670  -0.5547
480   진행   2.850427  16.388608   Topic3  -5.8176  -0.5808

[213 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.529089   가능
0         2  0.264545   가능
0         3  0.264545   가능
2         1  0.643300   감소
2         2  0.160825   감소
...     ...       ...  ...
566       2  0.202350   효과
566       3  0.404700   효과
923       1  0.689865   훈련
923       2  0.344932   훈련
923       3  0.344932   훈련

[412 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])